# Sea ice export

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
year_start = 2002
year_stop = 2020

## Define request

In [ ]:
time_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": ["01", "02", "03", "04", "10", "11", "12"],
}

requests = {
    "satellite-sea-ice-drift": time_request
    | {
        "variable": "all",
        "region": "northern_hemisphere",
        "day": [f"{day:02d}" for day in range(1, 32)],
        "version": "1_0",
    },
    "satellite-sea-ice-thickness": time_request
    | {
        "processing_level": "level_3",
        "satellite_mission": ["envisat", "cryosat_2"],
        "variable": ["sea_ice_thickness"],
        "temporal_resolution": ["monthly"],
        "version": "3_0",
    },
    "satellite-sea-ice-concentration": time_request
    | {
        "variable": "all",
        "version": "3_0",
        "sensor": "ssmis",
        "region": ["northern_hemisphere"],
        "cdr_type": ["cdr"],
        "temporal_aggregation": "daily",
        "day": [f"{day:02d}" for day in range(1, 32)],
    },
}

## Define functions to cache

In [ ]:
def regrid_and_resample(ds, coarsen, time_resample):
    if coarsen:
        ds = ds.coarsen(xc=3, yc=3).mean(keep_attrs=True)
    if time_resample:
        grouped = ds.resample(time="1MS")
        if "uncert_dX_and_dY" in ds:
            ds["uncert_dX_and_dY"] = ds["uncert_dX_and_dY"] ** 2
        ds = grouped.mean(keep_attrs=True).dropna("time", how="all")

        sum_vars = set(ds.data_vars) & {"dX", "dY", "uncert_dX_and_dY"}
        if sum_vars:
            ds_sum = grouped.sum(keep_attrs=True).dropna("time", how="all")
            if "uncert_dX_and_dY" in sum_vars:
                ds["uncert_dX_and_dY"] = ds_sum["uncert_dX_and_dY"] ** (1 / 2)
                ds_sum = ds_sum.drop("uncert_dX_and_dY")
            ds = ds.merge(
                ds_sum.rename({var: f"{var}_sum" for var in ds_sum.data_vars})
            )
    return ds

## Download and transform

In [ ]:
datasets = {}
for collection_id, request in requests.items():
    print(f"{collection_id = }")
    datasets[collection_id] = download.download_and_transform(
        collection_id,
        request,
        chunks={"year": 1},
        transform_func=regrid_and_resample,
        transform_func_kwargs={
            "coarsen": collection_id != "satellite-sea-ice-drift",
            "time_resample": collection_id != "satellite-sea-ice-thickness",
        },
    )
common_times = xr.merge([ds[["time"]] for ds in datasets.values()], join="inner")
datasets = {
    collection_id: ds.sel(time=common_times["time"])
    for collection_id, ds in datasets.items()
}

## Plot uncertainty

In [ ]:
da = diagnostics.spatial_weighted_mean(
    datasets["satellite-sea-ice-drift"]["uncert_dX_and_dY"]
)
da = da.resample(time="1MS").mean()
da.plot()
for year in range(year_start, year_stop + 1):
    plt.axvspan(
        pd.to_datetime(f"{year}-04"),
        pd.to_datetime(f"{year}-10"),
        color="lightgrey",
        alpha=0.25,
        zorder=0,
    )
plt.grid()